# Programming differential equations in Julia

Julia has a rich ecosystem for modelling based on differential equations. The [SciML](https://sciml.ai/) project provides a convenient entry point.

The ecosystem provides different tools to define and solve differential equations, as well as explore the solutions obtained.

### Example 1

Consider the equation $$\frac{d}{dt}x(t) = \alpha x(t).$$

Given an initial condition $x(0) = x_0$, it can be verified that the solution to the initial value problem is given by $$x(t) = x_0 e^{\alpha t}.$$

In [ ]:
using ModelingToolkit, DifferentialEquations, Plots

@variables t x(t)=0.5 # specify symbolically the variables involved together with the initial condition
@parameters α=0.2 # define parameters

# the symbolic differentiation operator
D = Differential(t) 

# define the equation(s)
eqs = [D(x) ~ α*x]

# construct a symbolic system
@named sys = ODESystem(eqs, t)

# define the timespan of the simulation
tspan = (0.0, 10.0) 

# convert to ODE problem from symbolic form
prob = ODEProblem(sys, [], tspan) 

# Solve the ODE problem
sol = solve(prob) 

# play around with the result
plot(sol, title = "Linear ODE")

In [ ]:
sol(0) # value of solution at 0

In [ ]:
sol(10) # value of solution at 10

In [ ]:
sol # note the form of u (a vector of vectors), as well as the tabulation grid

In [ ]:
# here is how to tabulate over a pre-specified grid
sol = solve(prob, saveat = range(tspan[1], stop = tspan[2], length = 100)) 

A simpler (and perhaps slightly outdated) way of accomplishing the same.

In [ ]:
using DifferentialEquations

function lin_ode!(du, u, p, t)
    du[1] = p[1] * u[1]
end

u0 = [0.5]
p = [0.2]
tspan = (0.0, 10.0)  # Time interval for solution
npoints = 100        # Number of output points

prob1 = ODEProblem(lin_ode!, u0, tspan, p)
sol1 = solve(prob1, Tsit5()) # note that the solver has been explicitly specified here

plot(sol1)


In [ ]:
sol1

### Example 2 (the Lotka-Volterra predator-prey model, from the documentation)

We consider the interaction between a prey species with population density $x(t)$ and a predator species with population density $y(t)$. The parameter $\alpha$ is the birth rate of the prey species, $\gamma$ is the death rate of the predator species and the parameters $\beta$ and $\delta$ describe the effects of interaction between species.

$$ \frac{dx}{dt} = \alpha x - \beta x y $$
$$ \frac{dy}{dt} = -\gamma y + \delta x y $$

In [ ]:
using ModelingToolkit, DifferentialEquations, Plots

# Define our state variables: state(t) = initial condition
@variables t x(t)=1 y(t)=1 z(t)=2

# Define our parameters
@parameters α=1.5 β=1.0 γ=3.0 δ=1.0

# Define our differential: takes the derivative with respect to `t`
D = Differential(t)

# Define the differential equations
# Note the last element, which is an algebraic equation counting the total number of animals
eqs = [D(x) ~ α * x - β * x * y
       D(y) ~ -γ * y + δ * x * y
       z ~ x + y] 

# Bring these pieces together into an ODESystem with independent variable t
@named sys = ODESystem(eqs, t)

# Symbolically Simplify the System (removes the algebraic equation)
simpsys = structural_simplify(sys)

# Convert from a symbolic to a numerical problem to simulate
tspan = (0.0, 10.0)
prob = ODEProblem(simpsys, [], tspan)

# Solve the ODE
sol = solve(prob)

# Plot the solution
p1 = plot(sol, title = "Rabbits vs Wolves")
p2 = plot(sol, idxs = z, title = "Total Animals")

plot(p1, p2, layout = (2, 1))